# 📚 FSKU_2_모델학습

## 📋 노트북 개요

### 목적
이 노트북은 FSKU 금융 AI Challenge를 위한 모델 학습 단계입니다. `FSKU_1_데이터증강_RAG.ipynb`에서 생성된 증강 데이터를 사용하여 금융 전문 AI 모델을 학습합니다.

### 입력 데이터
- **위치**: `data/augmented/` 폴더
- **형식**: JSON 파일 (questions_*.json)
- **내용**: 증강된 금융 관련 질문-답변 쌍

### 출력물
- **학습된 모델**: `models/fsku_finetuned_model/` (추론 단계에서 사용)
- **체크포인트**: `models/checkpoints/`
- **학습 메트릭**: `results/training_metrics.json`

### 핵심 제약사항
- RTX 4090 24GB 메모리 제한
- 단일 LLM만 사용 (앙상블 불가)
- 오프라인 환경에서 실행 가능
- 270분 내 515문항 처리 가능한 속도

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필수 라이브러리 설치 확인 및 자동 설치
import subprocess
import sys
import importlib.util

def install_package(package):
    """
    패키지가 설치되어 있지 않으면 자동으로 설치
    
    Args:
        package: 설치할 패키지 이름 (버전 포함 가능)
    """
    package_name = package.split('>')[0].split('=')[0].split('[')[0]
    
    # 더 효율적인 패키지 확인 방법
    if importlib.util.find_spec(package_name) is None:
        print(f"📦 {package} 설치 중...")
        try:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", package, "-q"],
                timeout=300  # 5분 타임아웃
            )
            print(f"✅ {package} 설치 완료!")
        except subprocess.TimeoutExpired:
            print(f"⚠️ {package} 설치 시간 초과 - 수동 설치 필요")
        except subprocess.CalledProcessError as e:
            print(f"❌ {package} 설치 실패: {e}")

# 필수 패키지 목록
required_packages = [
    "transformers>=4.36.0",
    "peft>=0.7.0",
    "bitsandbytes>=0.41.0",
    "accelerate>=0.25.0",
    "datasets",
    "sentencepiece",
    "protobuf",
    "scipy",
    "scikit-learn",
    "matplotlib",
    "seaborn",
    "psutil",  # 시스템 모니터링용
    "tensorboard",  # 학습 모니터링용
    "tqdm>=4.65.0"  # 진행 표시줄
]

print("🔍 필수 패키지 확인 중...")
for package in required_packages:
    install_package(package)
print("\n✅ 모든 필수 패키지가 준비되었습니다!")

In [ ]:
# 기본 라이브러리 임포트
import os
import json
import glob
import random
import warnings
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
from tqdm import tqdm

# 딥러닝 관련 라이브러리
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Hugging Face 라이브러리
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    logging as transformers_logging
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from datasets import Dataset as HFDataset

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 메시지 설정
warnings.filterwarnings('ignore')
transformers_logging.set_verbosity_error()

# 한글 폰트 설정 (시각화용)
import platform

if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# 시스템 리소스 모니터링
import psutil
print(f"\n💻 시스템 정보:")
print(f"  - CPU 코어: {psutil.cpu_count(logical=False)}개 (논리: {psutil.cpu_count()}개)")
print(f"  - RAM: {psutil.virtual_memory().total / 1024**3:.1f} GB")
print(f"  - 사용 가능 RAM: {psutil.virtual_memory().available / 1024**3:.1f} GB")

print("\n✅ 라이브러리 임포트 완료!")
print(f"📊 PyTorch 버전: {torch.__version__}")
print(f"🖥️ CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# 시드 고정 (재현성을 위해)
def set_seed(seed: int = 42):
    """
    모든 랜덤 시드를 고정하여 재현 가능한 결과를 보장
    
    Args:
        seed: 랜덤 시드 값
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print("🎲 랜덤 시드 고정 완료 (seed=42)")

## 2. 데이터 로딩 및 전처리

In [ ]:
# 데이터 로딩 함수 (메모리 효율성 개선)
def load_augmented_data(
    data_dir: str = "data/augmented", 
    max_samples: Optional[int] = None,
    min_answer_length: int = 10,
    max_answer_length: int = 4096
) -> List[Dict]:
    """
    증강된 데이터를 메모리 효율적으로 로드하여 반환
    
    Args:
        data_dir: 증강 데이터가 저장된 디렉토리 경로
        max_samples: 최대 로드할 샘플 수 (메모리 제한시 사용)
        min_answer_length: 최소 답변 길이
        max_answer_length: 최대 답변 길이
        
    Returns:
        모든 질문-답변 쌍이 담긴 리스트
    """
    all_data = []
    
    # JSON 파일 패턴으로 모든 증강 데이터 파일 찾기
    json_files = sorted(glob.glob(os.path.join(data_dir, "questions_*.json")))
    
    if not json_files:
        print(f"⚠️ 경고: {data_dir}에서 증강 데이터를 찾을 수 없습니다!")
        print("💡 먼저 FSKU_1_데이터증강_RAG.ipynb를 실행하여 데이터를 생성하세요.")
        return []
    
    print(f"📂 발견된 데이터 파일 수: {len(json_files)}개")
    
    # 데이터 품질 통계
    total_items = 0
    valid_items = 0
    duplicate_items = 0
    seen_questions = set()
    quality_stats = {
        'too_short': 0,
        'too_long': 0,
        'invalid_format': 0,
        'empty_fields': 0
    }
    
    # 각 파일에서 데이터 로드 (메모리 효율적 처리)
    failed_files = []
    with tqdm(total=len(json_files), desc="데이터 파일 로딩") as pbar:
        for file_path in json_files:
            if max_samples and len(all_data) >= max_samples:
                print(f"\nℹ️ 최대 샘플 수({max_samples})에 도달하여 로딩 중단")
                break
                
            try:
                # 메모리 효율적인 청크 단위 로딩
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    total_items += len(data)
                    
                    # 데이터 검증 및 중복 제거
                    for item in data:
                        if max_samples and len(all_data) >= max_samples:
                            break
                            
                        # 필수 필드 검증
                        if not all(k in item for k in ['question', 'answer']):
                            quality_stats['invalid_format'] += 1
                            continue
                            
                        # 빈 문자열 및 길이 검증
                        question = item['question'].strip()
                        answer = item['answer'].strip()
                        
                        if not question or not answer:
                            quality_stats['empty_fields'] += 1
                            continue
                        
                        # 답변 길이 검증
                        if len(answer) < min_answer_length:
                            quality_stats['too_short'] += 1
                            continue
                        
                        if len(answer) > max_answer_length:
                            quality_stats['too_long'] += 1
                            answer = answer[:max_answer_length] + "..."
                            
                        # 중복 검사 (해시 기반으로 개선)
                        question_hash = hash(question)
                        if question_hash in seen_questions:
                            duplicate_items += 1
                            continue
                            
                        seen_questions.add(question_hash)
                        valid_items += 1
                        
                        # 데이터 정규화
                        all_data.append({
                            'question': question,
                            'answer': answer,
                            'question_type': item.get('question_type', 'general'),
                            'source_file': os.path.basename(file_path),
                            'answer_length': len(answer)  # 통계용
                        })
                        
            except json.JSONDecodeError as e:
                print(f"\n❌ JSON 파싱 오류: {os.path.basename(file_path)}")
                failed_files.append(file_path)
            except MemoryError:
                print(f"\n⚠️ 메모리 부족! 현재까지 로드된 데이터만 사용합니다.")
                break
            except Exception as e:
                print(f"\n❌ 파일 로드 실패: {os.path.basename(file_path)} - {str(e)}")
                failed_files.append(file_path)
            
            pbar.update(1)
    
    # 로딩 통계 출력 (더 상세한 품질 통계)
    print(f"\n📊 데이터 로딩 통계:")
    print(f"  - 전체 항목: {total_items:,}개")
    print(f"  - 유효 항목: {valid_items:,}개 ({valid_items/max(total_items,1)*100:.1f}%)")
    print(f"  - 중복 제거: {duplicate_items:,}개")
    print(f"  - 품질 필터링:")
    print(f"    • 너무 짧음 (<{min_answer_length}자): {quality_stats['too_short']:,}개")
    print(f"    • 너무 김 (>{max_answer_length}자): {quality_stats['too_long']:,}개")
    print(f"    • 형식 오류: {quality_stats['invalid_format']:,}개")
    print(f"    • 빈 필드: {quality_stats['empty_fields']:,}개")
    print(f"  - 최종 로드: {len(all_data):,}개")
    
    if failed_files:
        print(f"  - 실패 파일: {len(failed_files)}개")
    
    # 메모리 사용량 추정
    estimated_memory = sys.getsizeof(all_data) / (1024**2)
    print(f"  - 예상 메모리: {estimated_memory:.1f} MB")
    
    # 데이터 품질 경고
    if len(all_data) < 1000:
        print("\n⚠️ 경고: 학습 데이터가 1,000개 미만입니다. 더 많은 데이터 생성을 권장합니다.")
    
    return all_data

# 데이터 로드
raw_data = load_augmented_data()

In [ ]:
# 데이터 통계 분석
def analyze_data(data: List[Dict]) -> None:
    """
    로드된 데이터의 통계 정보를 분석하고 출력
    
    Args:
        data: 분석할 데이터 리스트
    """
    if not data:
        print("❌ 분석할 데이터가 없습니다!")
        return
    
    # 기본 통계
    print("📊 데이터 통계 분석")
    print("=" * 50)
    print(f"총 데이터 수: {len(data):,}개")
    
    # 질문 유형별 분포
    question_types = {}
    answer_lengths = []
    
    for item in data:
        # 질문 유형 카운트
        q_type = item.get('question_type', 'unknown')
        question_types[q_type] = question_types.get(q_type, 0) + 1
        
        # 답변 길이 수집
        answer = item.get('answer', '')
        answer_lengths.append(len(answer))
    
    # 질문 유형 출력
    print("\n📝 질문 유형별 분포:")
    for q_type, count in sorted(question_types.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / len(data)) * 100
        print(f"  - {q_type}: {count:,}개 ({percentage:.1f}%)")
    
    # 답변 길이 통계
    if answer_lengths:
        print(f"\n📏 답변 길이 통계:")
        print(f"  - 평균: {np.mean(answer_lengths):.0f}자")
        print(f"  - 중간값: {np.median(answer_lengths):.0f}자")
        print(f"  - 최소: {np.min(answer_lengths)}자")
        print(f"  - 최대: {np.max(answer_lengths)}자")
        print(f"  - 표준편차: {np.std(answer_lengths):.0f}자")
        
        # 이상치 검출
        q1 = np.percentile(answer_lengths, 25)
        q3 = np.percentile(answer_lengths, 75)
        iqr = q3 - q1
        outliers = sum(1 for l in answer_lengths if l < q1 - 1.5*iqr or l > q3 + 1.5*iqr)
        if outliers > 0:
            print(f"  - 이상치: {outliers}개 ({outliers/len(answer_lengths)*100:.1f}%)")
    
    # 샘플 데이터 출력
    print("\n🔍 샘플 데이터 (첫 3개):")
    for i, item in enumerate(data[:3]):
        print(f"\n[샘플 {i+1}]")
        print(f"질문: {item.get('question', '')[:100]}...")
        print(f"답변: {item.get('answer', '')[:100]}...")
        print(f"유형: {item.get('question_type', 'unknown')}")

# 데이터 분석 실행
if raw_data:
    analyze_data(raw_data)

In [ ]:
# 학습/검증 데이터 분할 (층화 샘플링 지원)
def split_data(
    data: List[Dict], 
    test_size: float = 0.2, 
    seed: int = 42,
    stratify_by: Optional[str] = 'question_type'
) -> Tuple[List[Dict], List[Dict]]:
    """
    데이터를 학습용과 검증용으로 분할 (층화 샘플링 지원)
    
    Args:
        data: 전체 데이터 리스트
        test_size: 검증 데이터 비율 (기본값: 0.2)
        seed: 랜덤 시드
        stratify_by: 층화 샘플링 기준 필드 (None이면 단순 랜덤 분할)
        
    Returns:
        (학습 데이터, 검증 데이터) 튜플
    """
    if not data:
        return [], []
    
    random.seed(seed)
    
    # 층화 샘플링 시도
    if stratify_by and stratify_by in data[0]:
        # 카테고리별로 데이터 그룹화
        grouped_data = {}
        for item in data:
            key = item.get(stratify_by, 'unknown')
            if key not in grouped_data:
                grouped_data[key] = []
            grouped_data[key].append(item)
        
        train_data = []
        val_data = []
        
        # 각 카테고리별로 비율에 맞게 분할
        for category, items in grouped_data.items():
            random.shuffle(items)
            split_idx = int(len(items) * (1 - test_size))
            train_data.extend(items[:split_idx])
            val_data.extend(items[split_idx:])
        
        # 최종 셔플
        random.shuffle(train_data)
        random.shuffle(val_data)
        
        print(f"✂️ 층화 샘플링 분할 완료! (기준: {stratify_by})")
    else:
        # 단순 랜덤 분할
        shuffled_data = data.copy()
        random.shuffle(shuffled_data)
        
        # 분할 지점 계산
        split_idx = int(len(shuffled_data) * (1 - test_size))
        
        # 데이터 분할
        train_data = shuffled_data[:split_idx]
        val_data = shuffled_data[split_idx:]
        
        print(f"✂️ 랜덤 분할 완료!")
    
    print(f"  - 학습 데이터: {len(train_data):,}개 ({len(train_data)/len(data)*100:.1f}%)")
    print(f"  - 검증 데이터: {len(val_data):,}개 ({len(val_data)/len(data)*100:.1f}%)")
    
    # 카테고리 분포 확인
    if stratify_by:
        train_categories = {}
        val_categories = {}
        
        for item in train_data:
            cat = item.get(stratify_by, 'unknown')
            train_categories[cat] = train_categories.get(cat, 0) + 1
            
        for item in val_data:
            cat = item.get(stratify_by, 'unknown')
            val_categories[cat] = val_categories.get(cat, 0) + 1
        
        print(f"\n📊 카테고리 분포:")
        all_categories = set(train_categories.keys()) | set(val_categories.keys())
        for cat in sorted(all_categories):
            train_cnt = train_categories.get(cat, 0)
            val_cnt = val_categories.get(cat, 0)
            total_cnt = train_cnt + val_cnt
            if total_cnt > 0:
                print(f"  - {cat}: 학습 {train_cnt}개 ({train_cnt/total_cnt*100:.1f}%), 검증 {val_cnt}개 ({val_cnt/total_cnt*100:.1f}%)")
    
    return train_data, val_data

# 데이터 분할
if raw_data:
    train_data, val_data = split_data(raw_data, stratify_by='question_type')
else:
    train_data, val_data = [], []

## 3. 모델 선택 및 설정

In [ ]:
# 사용 가능한 모델 목록
AVAILABLE_MODELS = {
    "exaone": {
        "name": "LG-AI-EXAONE/EXAONE-3.0-7.8B-Instruct",
        "description": "LG AI Research의 한국어 특화 모델 (추천)",
        "size": "7.8B",
        "korean_specialized": True
    },
    "solar": {
        "name": "upstage/SOLAR-10.7B-v1.0",
        "description": "Upstage의 한국어 강화 모델",
        "size": "10.7B",
        "korean_specialized": True
    },
    "qwen": {
        "name": "Qwen/Qwen2.5-7B-Instruct",
        "description": "다국어 성능이 우수한 모델",
        "size": "7B",
        "korean_specialized": False
    },
    "llama-ko": {
        "name": "beomi/llama-2-ko-7b",
        "description": "한국어로 파인튜닝된 Llama 모델",
        "size": "7B",
        "korean_specialized": True
    }
}

print("🤖 사용 가능한 모델 목록:")
print("=" * 60)
for key, info in AVAILABLE_MODELS.items():
    print(f"\n[{key}] {info['name']}")
    print(f"  - 설명: {info['description']}")
    print(f"  - 크기: {info['size']}")
    print(f"  - 한국어 특화: {'✅' if info['korean_specialized'] else '❌'}")

In [ ]:
# 모델 선택 (환경변수로도 설정 가능)
import os

# 옵션: "exaone", "solar", "qwen", "llama-ko"
SELECTED_MODEL = os.getenv('FSKU_MODEL', 'exaone')  # 환경변수 또는 기본값

# 모델 선택 검증
if SELECTED_MODEL not in AVAILABLE_MODELS:
    print(f"⚠️ 잘못된 모델 선택: {SELECTED_MODEL}")
    print(f"사용 가능한 모델: {list(AVAILABLE_MODELS.keys())}")
    SELECTED_MODEL = "exaone"  # 기본값으로 복원
    print(f"기본 모델로 변경: {SELECTED_MODEL}")

# 선택된 모델 정보
model_info = AVAILABLE_MODELS[SELECTED_MODEL]
MODEL_NAME = model_info["name"]

print(f"\n✅ 선택된 모델: {MODEL_NAME}")
print(f"   {model_info['description']}")

# 모델별 특수 설정
if SELECTED_MODEL == "exaone":
    # EXAONE 모델은 특별한 토큰 처리가 필요할 수 있음
    print("\n💡 EXAONE 모델 특수 설정 적용")
    USE_SPECIAL_TOKENS = True
elif SELECTED_MODEL == "solar":
    # SOLAR 모델은 긴 컨텍스트 처리에 강함
    print("\n💡 SOLAR 모델 특수 설정 적용")
    USE_SPECIAL_TOKENS = False
else:
    USE_SPECIAL_TOKENS = False

## 4. QLoRA 설정 및 모델 로딩

In [ ]:
# QLoRA를 위한 4bit 양자화 설정
def get_quantization_config():
    """
    RTX 4090 24GB에 최적화된 4bit 양자화 설정 반환
    
    Returns:
        BitsAndBytesConfig 객체
    """
    return BitsAndBytesConfig(
        load_in_4bit=True,  # 4bit 양자화 사용
        bnb_4bit_compute_dtype=torch.float16,  # 계산은 FP16으로
        bnb_4bit_use_double_quant=True,  # 이중 양자화로 메모리 추가 절약
        bnb_4bit_quant_type="nf4"  # NormalFloat4 양자화 (더 나은 성능)
    )

# LoRA 설정
def get_lora_config():
    """
    LoRA (Low-Rank Adaptation) 설정 반환
    
    Returns:
        LoraConfig 객체
    """
    # 모델별 타겟 모듈 설정 (더 세밀한 설정)
    if "qwen" in MODEL_NAME.lower():
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    elif "solar" in MODEL_NAME.lower():
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    elif "llama" in MODEL_NAME.lower():
        target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    elif "exaone" in MODEL_NAME.lower():
        # EXAONE 모델은 특별한 구조를 가질 수 있음
        target_modules = ["q_proj", "v_proj", "k_proj", "o_proj"]
    else:
        # 대부분의 모델에서 작동하는 기본 설정
        target_modules = ["q_proj", "v_proj", "k_proj", "o_proj"]
    
    return LoraConfig(
        r=16,  # LoRA rank (8~32 범위, 높을수록 표현력 증가)
        lora_alpha=32,  # LoRA scaling parameter (일반적으로 r*2)
        target_modules=target_modules,  # 적용할 모듈
        lora_dropout=0.1,  # Dropout 비율
        bias="none",  # Bias 학습 여부
        task_type=TaskType.CAUSAL_LM,  # 언어 모델링 태스크
    )

# 설정 생성
quantization_config = get_quantization_config()
lora_config = get_lora_config()

print("⚙️ QLoRA 설정 완료!")
print(f"  - 양자화: 4bit (NF4)")
print(f"  - LoRA rank: {lora_config.r}")
print(f"  - LoRA alpha: {lora_config.lora_alpha}")
print(f"  - 타겟 모듈: {lora_config.target_modules}")

In [ ]:
# 모델과 토크나이저 로딩
print(f"\n🚀 모델 로딩 시작: {MODEL_NAME}")
print("⏳ 첫 실행시 모델 다운로드로 시간이 걸릴 수 있습니다 (10-20GB)...")

try:
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,  # 일부 모델은 커스텀 코드 필요
        use_fast=True  # Fast tokenizer 사용 (더 빠름)
    )
    
    # 패딩 토큰 설정 (없는 경우)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("ℹ️ 패딩 토큰을 EOS 토큰으로 설정했습니다.")
    
    # GPU 메모리 확인
    if torch.cuda.is_available():
        free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)
        print(f"\n💾 사용 가능한 GPU 메모리: {free_memory / 1024**3:.1f} GB")
        
        # 메모리가 부족한 경우 경고
        if free_memory < 10 * 1024**3:  # 10GB 미만
            print("⚠️ GPU 메모리가 부족할 수 있습니다. batch_size를 줄이는 것을 권장합니다.")
    
    # 모델 로드 (4bit 양자화 적용)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quantization_config,
        device_map="auto",  # GPU에 자동 배치
        trust_remote_code=True,
        torch_dtype=torch.float16,  # FP16 사용
        low_cpu_mem_usage=True  # CPU 메모리 사용량 감소
    )
    
    # 학습을 위한 모델 준비
    model = prepare_model_for_kbit_training(model)
    
    # LoRA 적용
    model = get_peft_model(model, lora_config)
    
    # 모델 정보 출력
    print("\n✅ 모델 로딩 완료!")
    print(f"📊 학습 가능한 파라미터:")
    model.print_trainable_parameters()
    
except Exception as e:
    print(f"\n❌ 모델 로딩 실패: {str(e)}")
    print("💡 해결 방법:")
    print("  1. 인터넷 연결 확인")
    print("  2. Hugging Face 토큰 설정 확인")
    print("  3. GPU 메모리 확인")
    raise e

## 5. 데이터셋 토큰화

In [ ]:
# 프롬프트 템플릿 정의
def format_prompt(question: str, answer: str, is_training: bool = True) -> str:
    """
    질문과 답변을 모델 학습용 프롬프트로 포맷팅
    
    Args:
        question: 질문 텍스트
        answer: 답변 텍스트
        is_training: 학습용인지 여부 (학습시에는 답변 포함)
        
    Returns:
        포맷된 프롬프트 문자열
    """
    # 모델별 프롬프트 템플릿
    if "exaone" in MODEL_NAME.lower():
        # EXAONE 모델용 템플릿
        if is_training:
            prompt = f"[|시스템|]당신은 금융 전문 AI 어시스턴트입니다. 정확하고 전문적인 답변을 제공하세요.[|종료|]\n[|사용자|]{question}[|종료|]\n[|AI|]{answer}[|종료|]"
        else:
            prompt = f"[|시스템|]당신은 금융 전문 AI 어시스턴트입니다. 정확하고 전문적인 답변을 제공하세요.[|종료|]\n[|사용자|]{question}[|종료|]\n[|AI|]"
    elif "solar" in MODEL_NAME.lower():
        # SOLAR 모델용 템플릿
        system_prompt = "당신은 한국 금융 시장에 정통한 전문가입니다. 질문에 대해 정확하고 상세한 답변을 제공하세요."
        if is_training:
            prompt = f"### System:\n{system_prompt}\n\n### User:\n{question}\n\n### Assistant:\n{answer}"
        else:
            prompt = f"### System:\n{system_prompt}\n\n### User:\n{question}\n\n### Assistant:\n"
    else:
        # 기본 템플릿 (Qwen, Llama 등)
        if is_training:
            prompt = f"질문: {question}\n\n답변: {answer}"
        else:
            prompt = f"질문: {question}\n\n답변: "
    
    return prompt

# 샘플 프롬프트 확인
if train_data:
    sample = train_data[0]
    sample_prompt = format_prompt(sample['question'], sample['answer'])
    print("📝 프롬프트 템플릿 예시:")
    print("=" * 60)
    print(sample_prompt[:500] + "..." if len(sample_prompt) > 500 else sample_prompt)
    print("=" * 60)

In [ ]:
# 토큰화 함수
def tokenize_function(examples: Dict[str, List]) -> Dict[str, List]:
    """
    배치 단위로 데이터를 토큰화
    
    Args:
        examples: 배치 데이터 (questions, answers 포함)
        
    Returns:
        토큰화된 데이터 딕셔너리
    """
    # 프롬프트 생성 (에러 처리 추가)
    prompts = []
    for question, answer in zip(examples['question'], examples['answer']):
        # None 값 체크
        if question is None or answer is None:
            continue
        # 문자열 변환 및 공백 제거
        question = str(question).strip()
        answer = str(answer).strip()
        if question and answer:
            prompt = format_prompt(question, answer, is_training=True)
            prompts.append(prompt)
    
    # 유효한 프롬프트가 없는 경우 처리
    if not prompts:
        raise ValueError("유효한 프롬프트가 없습니다!")
    
    # 토큰화
    model_inputs = tokenizer(
        prompts,
        max_length=2048,  # 최대 토큰 길이
        padding="max_length",  # 최대 길이까지 패딩
        truncation=True,  # 긴 텍스트는 자르기
        return_tensors="pt"
    )
    
    # 레이블 설정 (input_ids와 동일하게, 패딩 부분은 -100으로)
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    
    # 패딩 토큰은 손실 계산에서 제외 (-100으로 설정)
    model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100
    
    return model_inputs

# 데이터를 HuggingFace Dataset으로 변환
def prepare_datasets(train_data: List[Dict], val_data: List[Dict]):
    """
    학습/검증 데이터를 HuggingFace Dataset 형식으로 변환하고 토큰화
    
    Args:
        train_data: 학습 데이터 리스트
        val_data: 검증 데이터 리스트
        
    Returns:
        (토큰화된 학습 데이터셋, 토큰화된 검증 데이터셋)
    """
    # 리스트를 DataFrame으로 변환 (쉬운 처리를 위해)
    train_df = pd.DataFrame(train_data)
    val_df = pd.DataFrame(val_data)
    
    # HuggingFace Dataset으로 변환
    train_dataset = HFDataset.from_pandas(train_df)
    val_dataset = HFDataset.from_pandas(val_df)
    
    # 토큰화 적용
    print("🔄 학습 데이터 토큰화 중...")
    tokenized_train = train_dataset.map(
        tokenize_function,
        batched=True,
        batch_size=32,
        remove_columns=train_dataset.column_names
    )
    
    print("🔄 검증 데이터 토큰화 중...")
    tokenized_val = val_dataset.map(
        tokenize_function,
        batched=True,
        batch_size=32,
        remove_columns=val_dataset.column_names
    )
    
    print("\n✅ 토큰화 완료!")
    print(f"  - 학습 데이터: {len(tokenized_train)}개")
    print(f"  - 검증 데이터: {len(tokenized_val)}개")
    
    return tokenized_train, tokenized_val

# 데이터셋 준비
if train_data and val_data:
    tokenized_train_dataset, tokenized_val_dataset = prepare_datasets(train_data, val_data)
else:
    print("⚠️ 학습 데이터가 없습니다! FSKU_1_데이터증강_RAG.ipynb를 먼저 실행하세요.")
    tokenized_train_dataset, tokenized_val_dataset = None, None

In [ ]:
# 토큰 길이 분포 시각화
def visualize_token_distribution(dataset, title="Token Length Distribution"):
    """
    데이터셋의 토큰 길이 분포를 시각화
    
    Args:
        dataset: 토큰화된 데이터셋
        title: 그래프 제목
    """
    if dataset is None:
        return
    
    # 실제 토큰 길이 계산 (패딩 제외) - 메모리 효율적으로
    lengths = []
    sample_size = min(1000, len(dataset))  # 최대 1000개 샘플만 분석
    indices = np.random.choice(len(dataset), sample_size, replace=False)
    
    for idx in indices:
        item = dataset[int(idx)]
        # attention_mask가 1인 부분만 실제 토큰
        actual_length = sum(item['attention_mask'])
        lengths.append(actual_length)
    
    print(f"\n📊 샘플 크기: {sample_size}개 (전체 {len(dataset)}개 중)")
    
    # 통계 계산
    avg_length = np.mean(lengths)
    median_length = np.median(lengths)
    max_length = np.max(lengths)
    
    # 시각화
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=50, alpha=0.7, color='blue', edgecolor='black')
    plt.axvline(avg_length, color='red', linestyle='--', label=f'Average: {avg_length:.0f}')
    plt.axvline(median_length, color='green', linestyle='--', label=f'Median: {median_length:.0f}')
    plt.xlabel('Token Length')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"📊 토큰 길이 통계:")
    print(f"  - 평균: {avg_length:.0f} 토큰")
    print(f"  - 중간값: {median_length:.0f} 토큰")
    print(f"  - 최대: {max_length} 토큰")
    print(f"  - 2048 토큰 초과: {sum(1 for l in lengths if l >= 2048)}개 ({sum(1 for l in lengths if l >= 2048)/len(lengths)*100:.1f}%)")

# 학습 데이터 토큰 분포 확인
if tokenized_train_dataset:
    visualize_token_distribution(tokenized_train_dataset, "Training Data Token Distribution")

## 6. 학습 설정

In [ ]:
# 학습 하이퍼파라미터 설정 (개선된 동적 설정)
def get_training_args(output_dir: str = "./models/checkpoints"):
    """
    RTX 4090 24GB에 최적화된 학습 설정 반환
    
    Args:
        output_dir: 체크포인트 저장 디렉토리
        
    Returns:
        TrainingArguments 객체
    """
    # 동적 배치 크기 계산 (더 정교한 메모리 관리)
    batch_size = 4  # 기본값
    gradient_accumulation = 2  # 기본값
    
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        allocated_memory = torch.cuda.memory_allocated(0) / 1024**3
        free_memory = total_memory - allocated_memory
        
        # 모델 크기에 따른 동적 배치 크기 조정
        if SELECTED_MODEL in AVAILABLE_MODELS:
            model_size = float(AVAILABLE_MODELS[SELECTED_MODEL]['size'].replace('B', ''))
        else:
            model_size = 7.0  # 기본값
        
        # 메모리와 모델 크기를 고려한 배치 크기 결정
        if free_memory < 8 or model_size > 10:
            batch_size = 1
            gradient_accumulation = 8
        elif free_memory < 12:
            batch_size = 2
            gradient_accumulation = 4
        elif free_memory < 16:
            batch_size = 3
            gradient_accumulation = 3
        else:
            batch_size = 4
            gradient_accumulation = 2
            
        print(f"💾 GPU 메모리: {free_memory:.1f}GB 여유")
        print(f"🤖 모델 크기: {model_size}B")
        print(f"⚙️ 배치 크기: {batch_size}, Gradient Accumulation: {gradient_accumulation}")
        print(f"   → 실효 배치 크기: {batch_size * gradient_accumulation}")
    
    # 전체 학습 스텝 수 계산
    if tokenized_train_dataset:
        steps_per_epoch = len(tokenized_train_dataset) // (batch_size * gradient_accumulation)
        num_epochs = 3
        total_steps = steps_per_epoch * num_epochs
        
        # 데이터가 적은 경우 에폭 수 증가
        if len(tokenized_train_dataset) < 5000:
            num_epochs = 5
            total_steps = steps_per_epoch * num_epochs
            print(f"ℹ️ 데이터가 적어 에폭을 {num_epochs}로 증가시켰습니다.")
    else:
        total_steps = 1000
        num_epochs = 3
    
    # 학습률 자동 조정
    base_lr = 2e-4
    if batch_size * gradient_accumulation < 8:
        # 작은 배치 크기에는 더 낮은 학습률
        learning_rate = base_lr * 0.5
    else:
        learning_rate = base_lr
    
    return TrainingArguments(
        # 기본 설정
        output_dir=output_dir,
        overwrite_output_dir=True,
        
        # 학습 설정
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation,
        gradient_checkpointing=True,  # 메모리 절약
        
        # 옵티마이저 설정
        learning_rate=learning_rate,
        weight_decay=0.01,
        adam_beta1=0.9,
        adam_beta2=0.999,
        adam_epsilon=1e-8,
        max_grad_norm=1.0,  # Gradient clipping
        optim="adamw_torch",  # 더 효율적인 옵티마이저
        
        # 학습률 스케줄러
        lr_scheduler_type="cosine",
        warmup_steps=int(total_steps * 0.1),  # 10% warmup
        warmup_ratio=0.0,  # warmup_steps 사용시 0으로 설정
        
        # 로깅 및 저장
        logging_steps=max(10, total_steps // 100),  # 최소 10 스텝, 최대 전체의 1%
        logging_first_step=True,
        save_strategy="steps",
        save_steps=max(100, total_steps // 10),  # 최소 100 스텝, 최대 10개 체크포인트
        save_total_limit=3,  # 최대 3개의 체크포인트만 유지
        save_safetensors=True,  # 더 안전한 형식으로 저장
        
        # 평가 설정
        evaluation_strategy="steps",
        eval_steps=max(100, total_steps // 10),
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        load_best_model_at_end=True,
        
        # 성능 최적화
        fp16=True,  # Mixed precision training
        fp16_opt_level="O1",  # 안정적인 mixed precision
        tf32=True,  # A100/4090에서 성능 향상
        dataloader_num_workers=min(4, psutil.cpu_count()),
        dataloader_pin_memory=True,  # GPU 전송 속도 향상
        remove_unused_columns=False,
        
        # 추가 설정
        push_to_hub=False,
        report_to=["tensorboard"],
        logging_dir="./logs",
        seed=42,
        
        # 디버깅 및 모니터링
        debug="underflow_overflow" if torch.cuda.is_available() else "",
        include_inputs_for_metrics=False,
    )

# 학습 설정 생성
training_args = get_training_args()

print("\n⚙️ 학습 설정 완료!")
print(f"  - 에폭 수: {training_args.num_train_epochs}")
print(f"  - 배치 크기: {training_args.per_device_train_batch_size} × {training_args.gradient_accumulation_steps} = {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - 학습률: {training_args.learning_rate:.2e}")
print(f"  - Warmup 스텝: {training_args.warmup_steps}")
print(f"  - 체크포인트 저장 간격: {training_args.save_steps} 스텝")
print(f"  - Mixed Precision: FP16={training_args.fp16}, TF32={training_args.tf32}")

In [ ]:
# 조기 종료 콜백 설정
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # 3번의 평가에서 개선이 없으면 종료
    early_stopping_threshold=0.001  # 최소 개선 폭
)

print("🛑 조기 종료 설정 완료!")
print(f"  - Patience: {early_stopping_callback.early_stopping_patience}")
print(f"  - Threshold: {early_stopping_callback.early_stopping_threshold}")

## 7. 모델 학습

In [ ]:
# 커스텀 Trainer 클래스 (향상된 모니터링)
class FSKUTrainer(Trainer):
    """
    FSKU 프로젝트용 커스텀 Trainer
    향상된 메트릭 추적과 메모리 관리 기능 포함
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_history = []
        self.perplexity_history = []
        self.gpu_memory_history = []
        self.learning_rate_history = []
        self.best_perplexity = float('inf')
        self.steps_since_improvement = 0
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        손실 계산 및 추가 메트릭 추적
        """
        # 기본 손실 계산
        outputs = model(**inputs)
        loss = outputs.loss
        
        # 손실 기록
        if loss is not None:
            self.loss_history.append(loss.item())
        
        # Perplexity 계산 및 모니터링
        if len(self.loss_history) % 50 == 0 and self.loss_history:
            avg_loss = np.mean(self.loss_history[-50:])
            perplexity = np.exp(min(avg_loss, 10))  # Overflow 방지
            self.perplexity_history.append(perplexity)
            
            # 개선 추적
            if perplexity < self.best_perplexity:
                self.best_perplexity = perplexity
                self.steps_since_improvement = 0
            else:
                self.steps_since_improvement += 50
            
            # GPU 메모리 모니터링
            if torch.cuda.is_available():
                gpu_memory = torch.cuda.memory_allocated() / 1024**3
                gpu_util = torch.cuda.memory_allocated() / torch.cuda.get_device_properties(0).total_memory * 100
                self.gpu_memory_history.append(gpu_memory)
                
                # 상태 출력
                print(f"\n📊 [스텝 {len(self.loss_history)}]")
                print(f"   Perplexity: {perplexity:.2f} (최고: {self.best_perplexity:.2f})")
                print(f"   GPU: {gpu_memory:.1f}GB ({gpu_util:.1f}%)")
                print(f"   개선 없음: {self.steps_since_improvement} 스텝")
                
                # 메모리 경고
                if gpu_util > 90:
                    print("   ⚠️ GPU 메모리 사용률 90% 초과!")
                
                # 학습 정체 경고
                if self.steps_since_improvement > 500:
                    print("   ⚠️ 500 스텝 이상 개선이 없습니다. 학습률 조정을 고려하세요.")
        
        return (loss, outputs) if return_outputs else loss
    
    def log(self, logs):
        """향상된 로깅"""
        # GPU 메트릭 추가
        if torch.cuda.is_available():
            logs["gpu_memory_gb"] = torch.cuda.memory_allocated() / 1024**3
            logs["gpu_utilization"] = torch.cuda.memory_allocated() / torch.cuda.get_device_properties(0).total_memory * 100
        
        # 학습률 추적
        if hasattr(self, 'lr_scheduler') and self.lr_scheduler is not None:
            current_lr = self.lr_scheduler.get_last_lr()[0]
            logs["learning_rate"] = current_lr
            self.learning_rate_history.append(current_lr)
        
        # Perplexity 추가
        if self.perplexity_history:
            logs["perplexity"] = self.perplexity_history[-1]
            
        super().log(logs)
    
    def _save_checkpoint(self, model, trial, metrics=None):
        """체크포인트 저장시 추가 정보 포함"""
        # 기본 체크포인트 저장
        super()._save_checkpoint(model, trial, metrics)
        
        # 추가 메트릭 저장
        checkpoint_folder = os.path.join(
            self.args.output_dir,
            f"{self.state.global_step}"
        )
        
        if os.path.exists(checkpoint_folder):
            metrics_file = os.path.join(checkpoint_folder, "training_metrics.json")
            additional_metrics = {
                "best_perplexity": float(self.best_perplexity) if self.best_perplexity != float('inf') else None,
                "steps_since_improvement": self.steps_since_improvement,
                "avg_gpu_memory": float(np.mean(self.gpu_memory_history)) if self.gpu_memory_history else None,
                "current_step": self.state.global_step,
                "timestamp": datetime.now().isoformat()
            }
            
            with open(metrics_file, 'w') as f:
                json.dump(additional_metrics, f, indent=2)

# Trainer 초기화
if tokenized_train_dataset and tokenized_val_dataset:
    trainer = FSKUTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False,  # Causal LM이므로 MLM 비활성화
            pad_to_multiple_of=8  # 효율성을 위해 8의 배수로 패딩
        ),
        callbacks=[early_stopping_callback],
    )
    
    print("\n✅ Trainer 초기화 완료!")
    
    # 예상 학습 시간 계산
    total_steps = len(tokenized_train_dataset) // (trainer.args.per_device_train_batch_size * trainer.args.gradient_accumulation_steps) * trainer.args.num_train_epochs
    estimated_time = total_steps * 0.5 / 60  # 스텝당 0.5초 가정, 분 단위
    
    print(f"📊 학습 정보:")
    print(f"  - 총 학습 스텝: {total_steps:,}")
    print(f"  - 예상 시간: 약 {estimated_time:.0f}분")
    print(f"  - 체크포인트 수: 약 {total_steps // trainer.args.save_steps}개")
else:
    trainer = None
    print("⚠️ 학습 데이터가 없어 Trainer를 초기화할 수 없습니다!")

In [ ]:
# GPU 메모리 정리 함수 (개선된 버전)
def clear_gpu_memory():
    """
    GPU 메모리를 효과적으로 정리하여 OOM 방지
    """
    import gc
    
    if torch.cuda.is_available():
        # 메모리 사용 전 상태
        before_allocated = torch.cuda.memory_allocated() / 1024**3
        before_reserved = torch.cuda.memory_reserved() / 1024**3
        
        # 모든 캐시된 메모리 해제
        torch.cuda.synchronize()
        
        # Python 가비지 컬렉션
        gc.collect()
        
        # CUDA 캐시 정리
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        
        # 메모리 사용 후 상태
        after_allocated = torch.cuda.memory_allocated() / 1024**3
        after_reserved = torch.cuda.memory_reserved() / 1024**3
        
        # 정리 결과 출력
        print(f"🧹 GPU 메모리 정리 완료!")
        print(f"   할당 메모리: {before_allocated:.2f} → {after_allocated:.2f} GB")
        print(f"   예약 메모리: {before_reserved:.2f} → {after_reserved:.2f} GB")
        print(f"   해제된 메모리: {(before_reserved - after_reserved):.2f} GB")
        
        # 사용 가능 메모리 계산
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        free_memory = total_memory - after_reserved
        print(f"   사용 가능: {free_memory:.2f} GB / {total_memory:.2f} GB")
        
        # 메모리 부족 경고 및 대응 방안
        if free_memory < 5:
            print("\n⚠️ GPU 메모리가 5GB 미만입니다!")
            print("💡 해결 방법:")
            print("   1. batch_size를 1로 줄이기")
            print("   2. gradient_accumulation_steps를 8-16으로 증가")
            print("   3. max_length를 1024로 감소")
            print("   4. gradient_checkpointing=True 확인")
            print("   5. 다른 GPU 프로세스 종료")
            
            # nvidia-smi 정보 제안
            print("\n💡 다른 프로세스 확인: nvidia-smi")
            
        return free_memory
    else:
        print("ℹ️ GPU를 사용할 수 없습니다.")
        return 0

# 학습 전 메모리 정리
free_memory = clear_gpu_memory()

# 메모리가 부족한 경우 경고
if free_memory > 0 and free_memory < 3:
    print("\n⚠️ 심각한 메모리 부족! 학습을 시작하기 전에 다음을 확인하세요:")
    print("   - 다른 노트북이나 프로그램 종료")
    print("   - 커널 재시작 고려")
    print("   - batch_size=1로 설정")

In [ ]:
# 모델 학습 실행 (향상된 오류 처리)
if trainer is not None:
    print("\n🚀 모델 학습을 시작합니다!")
    print("⏱️ 예상 소요 시간: 데이터 크기에 따라 1-4시간")
    print("💡 팁: 학습 중 GPU 메모리가 부족하면 batch_size를 줄이세요.")
    print("📊 TensorBoard 모니터링: tensorboard --logdir ./logs\n")
    
    # 학습 시작 시간 기록
    start_time = datetime.now()
    
    # 학습 진행 상태 추적
    training_successful = False
    error_count = 0
    max_retries = 2
    
    while not training_successful and error_count < max_retries:
        try:
            # 학습 실행
            train_result = trainer.train()
            training_successful = True
            
            # 학습 완료
            end_time = datetime.now()
            training_time = end_time - start_time
            
            print(f"\n✅ 학습 완료!")
            print(f"⏱️ 총 학습 시간: {training_time}")
            print(f"📊 최종 학습 손실: {train_result.training_loss:.4f}")
            
            # 학습 메트릭 저장 (더 상세한 정보)
            metrics = {
                "training_loss": float(train_result.training_loss),
                "training_time": str(training_time),
                "training_time_seconds": training_time.total_seconds(),
                "model_name": MODEL_NAME,
                "model_type": SELECTED_MODEL,
                "total_steps": train_result.global_step,
                "epochs": training_args.num_train_epochs,
                "train_samples": len(train_data) if 'train_data' in globals() else 0,
                "val_samples": len(val_data) if 'val_data' in globals() else 0,
                "batch_size": training_args.per_device_train_batch_size,
                "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
                "effective_batch_size": training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps,
                "learning_rate": float(training_args.learning_rate),
                "lora_rank": lora_config.r,
                "timestamp": datetime.now().isoformat(),
                "retry_count": error_count
            }
            
            # 커스텀 트레이너 메트릭 추가
            if hasattr(trainer, 'best_perplexity'):
                metrics["best_perplexity"] = float(trainer.best_perplexity) if trainer.best_perplexity != float('inf') else None
            
            if hasattr(trainer, 'gpu_memory_history') and trainer.gpu_memory_history:
                metrics["avg_gpu_memory_gb"] = float(np.mean(trainer.gpu_memory_history))
                metrics["max_gpu_memory_gb"] = float(np.max(trainer.gpu_memory_history))
            
            # 최종 검증 손실 추가
            if hasattr(trainer.state, 'best_metric'):
                metrics["best_eval_loss"] = float(trainer.state.best_metric)
            
            # 메트릭 저장
            os.makedirs("results", exist_ok=True)
            with open("results/training_metrics.json", "w", encoding="utf-8") as f:
                json.dump(metrics, f, indent=2, ensure_ascii=False)
            
            print("\n📊 학습 메트릭이 results/training_metrics.json에 저장되었습니다.")
            
            # 성능 요약
            if "best_perplexity" in metrics and metrics["best_perplexity"]:
                print(f"\n🏆 최고 성능:")
                print(f"  - Perplexity: {metrics['best_perplexity']:.2f}")
            if "best_eval_loss" in metrics:
                print(f"  - 검증 손실: {metrics['best_eval_loss']:.4f}")
            if "avg_gpu_memory_gb" in metrics:
                print(f"  - 평균 GPU 사용: {metrics['avg_gpu_memory_gb']:.1f} GB")
                
        except torch.cuda.OutOfMemoryError as e:
            error_count += 1
            print(f"\n❌ GPU 메모리 부족 오류! (시도 {error_count}/{max_retries})")
            
            # 메모리 정리
            clear_gpu_memory()
            
            if error_count < max_retries:
                print("\n💡 자동 복구 시도 중...")
                print("   배치 크기를 줄이고 다시 시도합니다.")
                
                # 배치 크기 감소
                trainer.args.per_device_train_batch_size = max(1, trainer.args.per_device_train_batch_size // 2)
                trainer.args.gradient_accumulation_steps = min(16, trainer.args.gradient_accumulation_steps * 2)
                
                print(f"   새로운 배치 크기: {trainer.args.per_device_train_batch_size}")
                print(f"   새로운 Gradient Accumulation: {trainer.args.gradient_accumulation_steps}")
                
                # 잠시 대기
                import time
                time.sleep(5)
            else:
                print("\n💡 수동 해결 방법:")
                print("  1. 커널 재시작 후 batch_size=1로 재실행")
                print("  2. gradient_accumulation_steps를 8~16으로 증가")
                print("  3. max_length를 1024로 감소")
                print("  4. 더 작은 모델 사용 고려")
                raise e
                
        except KeyboardInterrupt:
            print("\n⚠️ 사용자가 학습을 중단했습니다.")
            print("💡 현재까지의 체크포인트는 저장되었습니다.")
            training_successful = True  # 루프 종료
            
        except Exception as e:
            error_count += 1
            print(f"\n❌ 학습 중 오류 발생: {str(e)} (시도 {error_count}/{max_retries})")
            
            if error_count < max_retries:
                print("💡 5초 후 재시도합니다...")
                import time
                time.sleep(5)
            else:
                print("\n💡 일반적인 해결 방법:")
                print("  1. 인터넷 연결 확인 (모델 다운로드)")
                print("  2. 디스크 공간 확인 (체크포인트 저장)")
                print("  3. 에러 메시지 검색")
                raise e
else:
    print("⚠️ Trainer가 초기화되지 않아 학습을 시작할 수 없습니다!")
    print("\n🔍 체크리스트:")
    print("   [ ] data/augmented/ 폴더에 JSON 파일이 있는지 확인")
    print("   [ ] GPU가 제대로 인식되는지 확인")
    print("   [ ] 필요한 패키지가 모두 설치되었는지 확인")

## 8. 학습 곡선 시각화

In [ ]:
# 학습 로그 시각화
def plot_training_history(trainer):
    """
    학습 과정의 손실 변화를 시각화
    
    Args:
        trainer: 학습이 완료된 Trainer 객체
    """
    if trainer is None or not hasattr(trainer.state, 'log_history'):
        print("⚠️ 학습 로그가 없습니다!")
        return
    
    # 로그에서 손실 값 추출
    log_history = trainer.state.log_history
    train_loss = []
    eval_loss = []
    steps = []
    
    for log in log_history:
        if 'loss' in log:
            train_loss.append(log['loss'])
            steps.append(log.get('step', len(train_loss)))
        if 'eval_loss' in log:
            eval_loss.append(log['eval_loss'])
    
    # 시각화
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 학습 손실
    if train_loss:
        ax1.plot(steps[:len(train_loss)], train_loss, 'b-', label='Training Loss')
        ax1.set_xlabel('Steps')
        ax1.set_ylabel('Loss')
        ax1.set_title('Training Loss Over Time')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
    
    # 검증 손실
    if eval_loss:
        eval_steps = [i * training_args.eval_steps for i in range(1, len(eval_loss) + 1)]
        ax2.plot(eval_steps, eval_loss, 'r-', marker='o', label='Validation Loss')
        ax2.set_xlabel('Steps')
        ax2.set_ylabel('Loss')
        ax2.set_title('Validation Loss Over Time')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 최종 손실 출력
    if train_loss:
        print(f"\n📊 최종 학습 손실: {train_loss[-1]:.4f}")
    if eval_loss:
        print(f"📊 최종 검증 손실: {eval_loss[-1]:.4f}")
        print(f"📊 최고 검증 손실: {min(eval_loss):.4f}")

# 학습 곡선 그리기
if trainer and hasattr(trainer, 'state'):
    plot_training_history(trainer)

## 9. 모델 저장

In [ ]:
# 최종 모델 저장
def save_final_model(trainer, output_dir: str = "models/fsku_finetuned_model"):
    """
    학습된 모델을 추론용으로 저장
    
    Args:
        trainer: 학습이 완료된 Trainer 객체
        output_dir: 모델 저장 경로
    """
    if trainer is None:
        print("⚠️ 저장할 모델이 없습니다!")
        return
    
    print(f"\n💾 모델을 저장합니다: {output_dir}")
    
    # 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # 최고 성능 모델 저장 (LoRA 어댑터만)
    trainer.save_model(output_dir)
    
    # 토크나이저도 저장
    trainer.tokenizer.save_pretrained(output_dir)
    
    # 설정 정보 저장 (더 상세한 정보)
    config_info = {
        "base_model": MODEL_NAME,
        "model_type": SELECTED_MODEL,
        "training_completed": datetime.now().isoformat(),
        "lora_config": {
            "r": lora_config.r,
            "lora_alpha": lora_config.lora_alpha,
            "lora_dropout": lora_config.lora_dropout,
            "target_modules": lora_config.target_modules
        },
        "training_args": {
            "num_train_epochs": training_args.num_train_epochs,
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
            "learning_rate": training_args.learning_rate,
            "warmup_steps": training_args.warmup_steps,
            "fp16": training_args.fp16
        },
        "dataset_info": {
            "train_samples": len(train_data) if 'train_data' in globals() else 0,
            "val_samples": len(val_data) if 'val_data' in globals() else 0
        },
        "final_loss": float(trainer.state.log_history[-1].get('loss', 0)) if hasattr(trainer, 'state') and trainer.state.log_history else 'N/A',
        "quantization": "4bit",
        "device": "cuda" if torch.cuda.is_available() else "cpu"
    }
    
    with open(os.path.join(output_dir, "training_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_info, f, indent=2, ensure_ascii=False)
    
    print("\n✅ 모델 저장 완료!")
    print(f"📁 저장 위치: {output_dir}")
    print("\n📝 저장된 파일:")
    print("  - adapter_model.safetensors (LoRA 가중치)")
    print("  - adapter_config.json (LoRA 설정)")
    print("  - tokenizer 파일들")
    print("  - training_config.json (학습 정보)")
    print("\n💡 추론시 이 경로를 FSKU_3_추론.ipynb에서 사용하세요!")
    
    return output_dir

# 모델 저장 실행
if trainer:
    saved_model_path = save_final_model(trainer)
else:
    print("⚠️ 학습된 모델이 없어 저장할 수 없습니다!")

## 10. 모델 평가 및 검증

In [ ]:
# 학습된 모델로 샘플 생성 테스트 (향상된 생성 파라미터)
def generate_sample(
    model, 
    tokenizer, 
    prompt: str, 
    max_length: int = 512,
    temperature: float = 0.8,
    top_p: float = 0.95,
    top_k: int = 50
):
    """
    학습된 모델로 텍스트 생성 테스트 (개선된 생성 파라미터)
    
    Args:
        model: 학습된 모델
        tokenizer: 토크나이저
        prompt: 입력 프롬프트
        max_length: 최대 생성 길이
        temperature: 생성 다양성 (0.1~1.0)
        top_p: nucleus sampling 파라미터
        top_k: top-k sampling 파라미터
        
    Returns:
        생성된 텍스트
    """
    # 모델을 평가 모드로
    model.eval()
    
    # 메모리 효율을 위한 캐시 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # 프롬프트 토큰화
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    
    # GPU로 이동
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # 텍스트 생성 (더 나은 생성 파라미터)
    with torch.no_grad():
        try:
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                do_sample=True,
                repetition_penalty=1.2,  # 반복 방지
                no_repeat_ngram_size=3,  # 3-gram 반복 방지
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                early_stopping=True,  # EOS 토큰 만나면 조기 종료
                num_beams=1,  # 빔 서치 비활성화 (더 빠른 생성)
            )
        except torch.cuda.OutOfMemoryError:
            print("⚠️ GPU 메모리 부족! 더 짧은 길이로 재시도...")
            torch.cuda.empty_cache()
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length // 2,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
    
    # 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 입력 프롬프트 제거
    if generated_text.startswith(prompt):
        response = generated_text[len(prompt):].strip()
    else:
        # 프롬프트가 포함되지 않은 경우
        response = generated_text.strip()
    
    return response

# 샘플 테스트
if trainer and val_data:
    print("\n🧪 학습된 모델 테스트")
    print("=" * 60)
    
    # 검증 데이터에서 샘플 선택
    num_samples = min(3, len(val_data))
    test_samples = random.sample(val_data, num_samples)
    
    # 다양한 온도로 테스트
    temperatures = [0.7, 0.8, 0.9]
    
    for i, sample in enumerate(test_samples):
        print(f"\n[테스트 {i+1}]")
        print(f"질문: {sample['question'][:200]}...")
        print(f"\n정답: {sample['answer'][:300]}..." if len(sample['answer']) > 300 else f"\n정답: {sample['answer']}")
        
        # 질문만으로 프롬프트 생성
        test_prompt = format_prompt(sample['question'], "", is_training=False)
        
        # 다양한 온도로 생성
        temp = temperatures[i % len(temperatures)]
        print(f"\n모델 생성 (temperature={temp}):")
        
        try:
            generated = generate_sample(
                model, 
                tokenizer, 
                test_prompt,
                temperature=temp,
                max_length=min(512, len(sample['answer']) + 100)
            )
            print(generated[:300] + "..." if len(generated) > 300 else generated)
        except Exception as e:
            print(f"생성 실패: {str(e)}")
        
        print("-" * 60)
    
    # 생성 품질 평가 팁
    print("\n💡 생성 품질 평가 기준:")
    print("  1. 답변의 관련성과 정확성")
    print("  2. 문장의 유창성과 일관성")
    print("  3. 금융 용어의 적절한 사용")
    print("  4. 답변 길이의 적절성")
else:
    print("⚠️ 테스트할 모델이나 데이터가 없습니다!")

In [ ]:
# 최종 요약 및 체크리스트
print("\n" + "=" * 60)
print("📊 FSKU 모델 학습 완료 요약")
print("=" * 60)

if trainer:
    # 학습 완료 정보
    print(f"\n✅ 모델: {MODEL_NAME}")
    print(f"✅ 모델 타입: {SELECTED_MODEL}")
    print(f"✅ 학습 데이터: {len(train_data):,}개")
    print(f"✅ 검증 데이터: {len(val_data):,}개")
    print(f"✅ 학습 에폭: {training_args.num_train_epochs}")
    print(f"✅ 배치 크기: {training_args.per_device_train_batch_size} × {training_args.gradient_accumulation_steps}")
    print(f"✅ 최종 모델 저장: models/fsku_finetuned_model/")
    
    # 성능 요약
    if hasattr(trainer, 'state') and trainer.state.log_history:
        final_train_loss = trainer.state.log_history[-1].get('loss', 'N/A')
        if isinstance(final_train_loss, float):
            print(f"\n📈 성능 지표:")
            print(f"  - 최종 학습 손실: {final_train_loss:.4f}")
            print(f"  - 최종 Perplexity: {np.exp(min(final_train_loss, 10)):.2f}")
            
        if hasattr(trainer, 'best_perplexity') and trainer.best_perplexity != float('inf'):
            print(f"  - 최고 Perplexity: {trainer.best_perplexity:.2f}")
    
    # 최적화 팁
    print(f"\n💡 추론 최적화 팁:")
    print(f"  1. 배치 처리로 속도 향상")
    print(f"  2. Temperature 0.7-0.8 사용")
    print(f"  3. Max tokens를 적절히 제한")
    print(f"  4. GPU 메모리 모니터링")
    
    # 다음 단계 체크리스트
    print(f"\n📝 다음 단계 체크리스트:")
    print(f"  ☐ FSKU_3_추론.ipynb 파일 열기")
    print(f"  ☐ 모델 경로 확인: 'models/fsku_finetuned_model'")
    print(f"  ☐ test.csv 파일 준비")
    print(f"  ☐ 추론 실행 (270분 내 완료 목표)")
    print(f"  ☐ 결과 검증 및 제출")
    
    # 유용한 명령어
    print(f"\n🔧 유용한 명령어:")
    print(f"  • TensorBoard 실행: tensorboard --logdir ./logs")
    print(f"  • 모델 크기 확인: du -sh models/fsku_finetuned_model/")
    print(f"  • GPU 모니터링: watch -n 1 nvidia-smi")
    print(f"  • 메모리 정리: torch.cuda.empty_cache()")
    
    # 트러블슈팅
    print(f"\n🚨 자주 발생하는 문제 해결:")
    print(f"  • OOM 오류: batch_size=1, gradient_accumulation_steps=16")
    print(f"  • 느린 추론: vLLM 또는 TGI 사용 고려")
    print(f"  • 품질 문제: 더 많은 데이터로 재학습")
    print(f"  • 토큰 초과: max_length 조정")
    
else:
    print("\n❌ 학습이 완료되지 않았습니다!")
    print("\n💡 해결 방법:")
    print("  1. FSKU_1_데이터증강_RAG.ipynb 실행하여 데이터 생성")
    print("  2. data/augmented/ 폴더 확인")
    print("  3. GPU 및 메모리 확인")
    print("  4. 이 노트북 재실행")
    
    print("\n🔍 디버깅 체크리스트:")
    print("  ☐ data/augmented/ 폴더에 JSON 파일 존재")
    print("  ☐ GPU 인식 (torch.cuda.is_available())")
    print("  ☐ 필수 패키지 설치")
    print("  ☐ 인터넷 연결 (모델 다운로드)")

print("\n" + "=" * 60)
print("🎯 FSKU 금융 AI Challenge - 모델 학습 단계 완료!")
print("=" * 60)

In [ ]:
# 학습 결과 종합 분석
def analyze_training_results(trainer, train_data, val_data):
    """
    학습 결과를 종합적으로 분석하고 개선 제안
    """
    print("\n" + "=" * 60)
    print("📊 학습 결과 종합 분석")
    print("=" * 60)
    
    if not trainer or not hasattr(trainer, 'state'):
        print("❌ 분석할 학습 결과가 없습니다.")
        return
    
    # 1. 데이터 분석
    print("\n1️⃣ 데이터 통계")
    print(f"  - 학습 데이터: {len(train_data):,}개")
    print(f"  - 검증 데이터: {len(val_data):,}개")
    print(f"  - 학습/검증 비율: {len(train_data)/(len(train_data)+len(val_data))*100:.1f}%")
    
    # 데이터 품질 평가
    if len(train_data) < 1000:
        print("  ⚠️ 학습 데이터가 부족합니다 (권장: 5,000개 이상)")
    elif len(train_data) < 5000:
        print("  ℹ️ 학습 데이터가 적당합니다 (권장: 5,000개 이상)")
    else:
        print("  ✅ 충분한 학습 데이터")
    
    # 2. 학습 과정 분석
    print("\n2️⃣ 학습 과정")
    if hasattr(trainer.state, 'log_history') and trainer.state.log_history:
        total_steps = trainer.state.global_step
        print(f"  - 총 학습 스텝: {total_steps:,}")
        print(f"  - 완료 에폭: {trainer.state.epoch:.1f}/{training_args.num_train_epochs}")
        
        # 손실 추세 분석
        losses = [log.get('loss', 0) for log in trainer.state.log_history if 'loss' in log]
        if losses:
            initial_loss = losses[0]
            final_loss = losses[-1]
            improvement = (initial_loss - final_loss) / initial_loss * 100
            
            print(f"  - 초기 손실: {initial_loss:.4f}")
            print(f"  - 최종 손실: {final_loss:.4f}")
            print(f"  - 개선율: {improvement:.1f}%")
            
            if improvement < 10:
                print("  ⚠️ 학습 개선이 미미합니다. 학습률 조정 필요")
            elif improvement < 30:
                print("  ℹ️ 적절한 학습 진행")
            else:
                print("  ✅ 우수한 학습 개선")
    
    # 3. 성능 지표
    print("\n3️⃣ 성능 지표")
    if hasattr(trainer, 'best_perplexity') and trainer.best_perplexity != float('inf'):
        print(f"  - 최고 Perplexity: {trainer.best_perplexity:.2f}")
        
        if trainer.best_perplexity < 10:
            print("  ✅ 우수한 언어 모델 성능")
        elif trainer.best_perplexity < 50:
            print("  ℹ️ 적절한 언어 모델 성능")
        else:
            print("  ⚠️ 개선이 필요한 성능")
    
    if hasattr(trainer.state, 'best_metric'):
        print(f"  - 최고 검증 손실: {trainer.state.best_metric:.4f}")
    
    # 4. 리소스 사용
    print("\n4️⃣ 리소스 사용")
    if hasattr(trainer, 'gpu_memory_history') and trainer.gpu_memory_history:
        avg_gpu = np.mean(trainer.gpu_memory_history)
        max_gpu = np.max(trainer.gpu_memory_history)
        print(f"  - 평균 GPU 메모리: {avg_gpu:.1f} GB")
        print(f"  - 최대 GPU 메모리: {max_gpu:.1f} GB")
        
        gpu_util = max_gpu / 24 * 100  # RTX 4090 24GB 기준
        if gpu_util > 90:
            print("  ⚠️ GPU 메모리 사용률이 매우 높습니다")
        elif gpu_util > 70:
            print("  ℹ️ GPU 메모리를 효율적으로 사용 중")
        else:
            print("  💡 배치 크기를 늘려도 됩니다")
    
    # 5. 개선 제안
    print("\n5️⃣ 개선 제안")
    suggestions = []
    
    # 데이터 관련
    if len(train_data) < 5000:
        suggestions.append("• 더 많은 학습 데이터 생성 (목표: 10,000개)")
    
    # 성능 관련
    if hasattr(trainer, 'best_perplexity') and trainer.best_perplexity > 30:
        suggestions.append("• 학습률 조정 또는 에폭 수 증가")
        suggestions.append("• LoRA rank 증가 (현재: 16 → 32)")
    
    # 효율성 관련
    if hasattr(trainer, 'gpu_memory_history') and trainer.gpu_memory_history:
        if np.max(trainer.gpu_memory_history) < 18:  # 24GB의 75%
            suggestions.append("• 배치 크기 증가 가능")
    
    if suggestions:
        for suggestion in suggestions:
            print(suggestion)
    else:
        print("✅ 현재 설정이 최적화되어 있습니다!")
    
    # 6. 다음 단계
    print("\n6️⃣ 다음 단계")
    print("  1. 모델 저장 확인: models/fsku_finetuned_model/")
    print("  2. 추론 노트북 실행: FSKU_3_추론.ipynb")
    print("  3. 성능 평가 및 제출")
    
    return {
        'train_samples': len(train_data),
        'val_samples': len(val_data),
        'best_perplexity': getattr(trainer, 'best_perplexity', None),
        'suggestions': suggestions
    }

# 학습 결과 분석 실행
if trainer and train_data and val_data:
    analysis_results = analyze_training_results(trainer, train_data, val_data)
else:
    print("⚠️ 분석할 학습 결과가 없습니다.")